In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e1/sample_submission.csv
/kaggle/input/playground-series-s6e1/train.csv
/kaggle/input/playground-series-s6e1/test.csv


In [3]:
#importing neccessary libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor



In [4]:
#Loading the dataset
import pandas as pd

train = pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")
test  = pd.read_csv("/kaggle/input/playground-series-s6e1/test.csv")

print(train.shape)
train.head()


(630000, 13)


,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [5]:
#Spliting dataset into input and output
X = train.drop(columns=["exam_score"])
y = train["exam_score"]


In [6]:
#Columns Type
num_cols = X.select_dtypes(include=["int64","float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

print("Numerical:", list(num_cols))
print("Categorical:", list(cat_cols))


Numerical: ['id', 'age', 'study_hours', 'class_attendance', 'sleep_hours']
Categorical: ['gender', 'course', 'internet_access', 'sleep_quality', 'study_method', 'facility_rating', 'exam_difficulty']


In [7]:
#Data Preprocessing Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)


In [8]:
#Model Building
model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=3,
    subsample=0.8,
    random_state=42
)


In [9]:
#Full Pipeline
pipe = Pipeline([
    ("prep", preprocessor),
    ("model", model)
])


In [ ]:
#K-Fold Cross Validation 
kf = KFold(n_splits=5, shuffle=True, random_state=42)

rmse_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    pipe.fit(X_train, y_train)
    val_pred = pipe.predict(X_val)
    
    rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    rmse_scores.append(rmse)
    
    print(f"Fold {fold+1} RMSE: {rmse:.4f}")

print("\nMean CV RMSE:", np.mean(rmse_scores))


Fold 1 RMSE: 8.8122


In [ ]:
#Train on full Data
pipe.fit(X, y)


In [ ]:
#Test Prediction
test_pred = pipe.predict(test)


In [ ]:
#submission Files
submission = pd.DataFrame({
    "id": test["id"],
    "exam_score": test_pred
})

submission.to_csv("submission.csv", index=False)
submission.head()

